# Device & Patch Extractors


In [21]:
%load_ext autoreload
%autoreload all
%env LOGGING_LEVEL=INFO

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: LOGGING_LEVEL=INFO


In [22]:
import typing

from termcolor import colored

from hbit import bootstrap, core, types
from hbit.extractors import device_extractors, patch_extractors

In [23]:
service_factory = bootstrap.ServicesFactory()
service_factory.add_db()
service_factory.add_requests()
service_factory.add_client()
service_factory.add_anthropic_models(requests_per_second=1)

registry = service_factory.registry
registry

[01/27/25 22:09:51] INFO     HTTP Request: POST http://localhost:8000/api/v1/login/access-token     ]8;id=429610;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=182649;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [24]:
db = registry.get_service(core.DatabaseService)
# model = registry.get_service(types.DefaultModel)
model = registry.get_service(types.CodeModel)
model

ChatAnthropic(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x12cdbe990>, model='claude-3-sonnet-20240229', temperature=0.0, anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [25]:
class QuestionAnswer(typing.NamedTuple):
    text: str
    patch_build: str | None
    device_identifier: str | None


texts = [
    QuestionAnswer(
        text="Write a summary of how secure is an iPhone XS running iOS 17.7.2? Are there any known vulnerabilities or security concerns with this version?",
        patch_build="21h221",
        device_identifier="iphone11,2",
    ),
    QuestionAnswer(
        text="How security is my iPhone 13 Pro device if I have patch 18.1.0 installed identified by build 22B83.",
        patch_build="22b83",
        device_identifier="iphone14,2",
    ),
    QuestionAnswer(
        text="What version does my patch with build 22B83 have?",
        patch_build="22b83",
        device_identifier=None,
    ),
    QuestionAnswer(
        text="Can you explain if iPhone 12 running iOS 16.5.1 is safe from known vulnerabilities?",
        patch_build="20f75",
        device_identifier="iphone13,2",
    ),
    QuestionAnswer(
        text="What security improvements are introduced in iOS 17.0.3 for the iPhone 14 Pro Max?",
        patch_build="21a360",
        device_identifier="iphone15,3",
    ),
    QuestionAnswer(
        text="Is my iPhone SE (2nd Gen) secure if it's updated to iOS 15.7.4 with build number 19H321?",
        patch_build="19h321",
        device_identifier=None,
    ),
    QuestionAnswer(
        text="Are there any critical security concerns with iOS 16.6.0 on the iPhone 11 Pro?",
        patch_build="20g75",
        device_identifier="iphone12,3",
    ),
    QuestionAnswer(
        text="What known bugs or vulnerabilities exist for iPhone XR devices running iOS 15.2.1?",
        patch_build="19c63",
        device_identifier="iphone11,8",
    ),
    QuestionAnswer(
        text="How does the iOS 17.1.1 patch improve security on iPhone 13 devices? What is its build number?",
        patch_build="21b91",
        device_identifier="iphone14,5",
    ),
    QuestionAnswer(
        text="Can you confirm if my iPhone XS Max running iOS 15.2 build 19C56 is fully secure?",
        patch_build="19c56",
        device_identifier="iphone11,6",
    ),
    QuestionAnswer(
        text="What's the latest iOS patch for iPhone 14, and does it address critical security vulnerabilities?",
        patch_build=None,
        device_identifier="iphone14,7",
    ),
    QuestionAnswer(
        text="Are there any concerns with iPhone 12 Mini running iOS 17.6.2 build 21G123?",
        patch_build=None,
        device_identifier="iphone13,1",
    ),
    QuestionAnswer(
        text="What issues might arise from using an iPhone 11?",
        patch_build=None,
        device_identifier="iphone12,1",
    ),
]

In [26]:
index = 8
text = texts[index].text
text

'How does the iOS 17.1.1 patch improve security on iPhone 13 devices? What is its build number?'

## SQL Device Extractor

In [17]:
sql_device_extractor = device_extractors.SqlDeviceExtractor(model=model, db=db)
sql_device_extractor

In [18]:
sql_device_extractor.extract_device_identifier(text)

[01/27/25 20:46:10] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=513052;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=171079;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

'iphone14,2'

## Structured Device Extractor


In [27]:
structured_device_extractor = device_extractors.StructureDeviceExtractor(
    model=model, db=db
)
structured_device_extractor

In [28]:
structured_device_extractor.extract_device_identifier(text)

1[{"variableName": "ID_TO_MEANING", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "builtins.dict"}, {"variableName": "NULL", "type": "unknown", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "_pydevd_bundle.pydevd_constants.Null"}]

1[{"variableName": "ID_TO_MEANING", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "builtins.dict"}, {"variableName": "NULL", "type": "unknown", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "_pydevd_bundle.pydevd_constants.Null"}]



[01/27/25 22:10:24] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=449772;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=756331;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

<IPython.core.display.Markdown object>


'iphone14,5'

## SQL Patch Extractor


In [50]:
sql_patch_extractor = patch_extractors.SqlPatchExtractor(model=model, db=db)
sql_patch_extractor

In [51]:
sql_patch_extractor.extract_patch_build(text)

[01/27/25 19:56:03] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=16258;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=57014;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

'21b91'

## Structured Patch Extractor


In [52]:
structured_patch_extractor = patch_extractors.StructurePatchExtractor(
    model=model, db=db
)
structured_patch_extractor

In [53]:
structured_patch_extractor.extract_patch_build(text)

[01/27/25 19:56:06] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=545470;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=980522;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

## Test all And Answers

In [54]:
for i, (text, patch_build, device_identifier) in enumerate(texts):
    print(
        f"================================== Example {i} =================================="
    )
    print(f"Text: {text}")

    structured_device_identifier = (
        structured_device_extractor.extract_device_identifier(text)
    )
    structured_color = (
        "green" if structured_device_identifier == device_identifier else "red"
    )
    sql_device_identifier = sql_device_extractor.extract_device_identifier(text)
    sql_color = "green" if sql_device_identifier == device_identifier else "red"

    print(
        f"Device Extractor: Expected: {colored(device_identifier, 'green')}, "
        f"Structured Response: {colored(structured_device_identifier, structured_color)}, "
        f"SQL Response: {colored(sql_device_identifier, sql_color)}"
    )

    structured_patch_build = structured_patch_extractor.extract_patch_build(text)
    structured_color = "green" if structured_patch_build == patch_build else "red"
    sql_patch_build = sql_patch_extractor.extract_patch_build(text)
    sql_color = "green" if sql_patch_build == patch_build else "red"

    print(
        f"Patch Extractor: Expected: {colored(patch_build, 'green')}, "
        f"Structured Response: {colored(structured_patch_build, structured_color)}, "
        f"SQL Response: {colored(sql_patch_build, sql_color)}"
    )
    print()

================================== Example 0 ==================================
Text: Write a summary of how secure is an iPhone XS running iOS 17.7.2? Are there any known vulnerabilities or security concerns with this version?


[01/27/25 19:57:48] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=802699;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=142569;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

[01/27/25 19:57:49] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=939804;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=881663;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Device Extractor: Expected: iphone11,2, Structured Response: None, SQL Response: iphone11,2


[01/27/25 19:57:51] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=374317;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=700927;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

...

[01/27/25 19:58:55] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=316701;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=874536;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Device Extractor: Expected: iphone12,1, Structured Response: iphone12,1, SQL Response: iphone12,1


[01/27/25 19:58:56] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=764883;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=996729;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

[01/27/25 19:58:57] INFO     HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 ]8;id=810765;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=639995;file:///Users/macbook/Projects/hbit/.venv/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Patch Extractor: Expected: None, Structured Response: None, SQL Response: None

